#### Llama-parseでPDFを読み込み、Markdown形式に保存してみる

In [77]:
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv()

True

In [51]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-large")
llm = OpenAI(model="gpt-3.5-turbo-0125")

Settings.llm = llm
Settings.embed_model = embed_model

#### Using LlamaParse PDF reader for PDF Parsing

1. raw Markdown textをインデックス構築のノードとして使用し、結果を生成するためのシンプルなクエリエンジンを適用する
2. MarkdownElementNodeParserを使用して、LlamaParse出力のMarkdown結果を解析し、生成用の再帰的な取得クエリエンジンを構築する

In [52]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data("1.pdf")

Started parsing the file under job_id ba057fc2-7184-4d9f-856e-2074ab7f740b
.

In [53]:
len(documents)

14

In [54]:
from copy import deepcopy
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex

def get_page_nodes(docs, separator="\n---\n"):
    """Split each document into page node, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split(separator)
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata)
            )
            nodes.append(node)
    return nodes

In [55]:
page_nodes = get_page_nodes(documents)

In [56]:
len(page_nodes)

14

In [57]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(
    llm=OpenAI(model="gpt-4o-mini"),
    num_workers=8
)

In [58]:
nodes = node_parser.get_nodes_from_documents(documents)

0it [00:00, ?it/s]

0it [00:00, ?it/s]
2it [00:00, 7219.11it/s]
0it [00:00, ?it/s]
2it [00:00, 1988.29it/s]
1it [00:00, 8256.50it/s]
1it [00:00, 7244.05it/s]
1it [00:00, 17848.10it/s]
2it [00:00, 18001.30it/s]
1it [00:00, 7145.32it/s]
0it [00:00, ?it/s]
3it [00:00, 6380.79it/s]
18it [00:00, 57151.76it/s]
6it [00:00, 60640.54it/s]
0it [00:00, ?it/s]


In [59]:
len(nodes)

125

In [60]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [61]:
base_nodes[1].get_content()

'経営理念・ブランドコンセプト\n\n ４℃ホールディングスグループの\n\n At a Glance\n\n Corporate Philosophy 経営理念\n\n私達は、お客様に信頼される企業を目指します。\n\n私達は、社員に夢を与える企業を目指します。\n\n私達は、社会に貢献できる企業を目指します。\n\n私達は、株主に期待される企業を目指します。\n\n Corporate Message コーポレートメッセージ\n\n当グループは、4℃ブランドを中心としたグローバルファッション創造企業として、お客様の一歩先のニーズに応える、お客様の生活文化を向上させる企業であり続けます。\n\n 業績（2024年月期）'

In [62]:
objects[1].get_content()

"This table presents key statistics related to a company's workforce, including the number of subsidiaries, employee training hours, total employees, female management ratio, training expenses, and performance against government gender equality targets.,\nwith the following columns:\n- 連結子会社数: Number of consolidated subsidiaries.\n- 年間教育研修: Total annual training hours.\n- 総時間: Total hours (not specified).\n- 従業員数（連結）: Total number of consolidated employees.\n- 女性管理職比率: Percentage of female managers.\n- 年間教育研修費: Annual training expenses.\n- 政府目標の「女性管理職比率 30%」を上回る: Indicates whether the company exceeds the government target for female management ratio.\n"

In [63]:
page_nodes

[TextNode(id_='60d9a38e-b137-4bba-8776-1958c17ffb30', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 統 合レポート\n\n# ４℃\n\n# レポート 2024\n\nすべては、お客様の “笑顔” や “ときめき” のために', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='0009b274-b3f7-42d7-81a1-7f568673e7c2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 経営理念・ブランドコンセプト\n\n# ４℃ホールディングスグループの\n\n# At a Glance\n\n# Corporate Philosophy 経営理念\n\n私達は、お客様に信頼される企業を目指します。\n\n私達は、社員に夢を与える企業を目指します。\n\n私達は、社会に貢献できる企業を目指します。\n\n私達は、株主に期待される企業を目指します。\n\n# Corporate Message コーポレートメッセージ\n\n当グループは、4℃ブランドを中心としたグローバルファッション創造企業として、お客様の一歩先のニーズに応える、お客様の生活文化を向上させる企業であり続けます。\n\n# 業績（2024年月期）\n\n|売上高|394.5億

In [64]:
# dump both indexed tables and page text into the vector index
recursive_index = VectorStoreIndex(nodes=base_nodes + objects + page_nodes)

In [65]:
print(page_nodes[12].get_content())

# データセクション

# 財務情報

# 主要財務データ

# 6カ年の推移

|項目|2019年2月期|2020年2月期|2021年2月期|2022年2月期|2023年2月期|2024年2月期|
|---|---|---|---|---|---|---|
|売上高 （百万円）|47,118|44,970|39,449|38,123|39,508|39,457|
|売上総利益 （百万円）|27,155|25,780|21,294|19,672|19,727|19,530|
|販売費及び一般管理費 （百万円）|22,171|21,804|18,527|17,884|17,748|17,433|
|営業利益 （百万円）|4,984|3,975|2,767|1,788|1,979|2,096|
|経常利益 （百万円）|6,804|4,312|3,195|2,293|2,342|2,515|
|親会社株主に帰属する当期純利益 （百万円）|2,440|2,475|1,622|1,490|1,149|1,300|

# 財政状態

|項目|2019年2月期|2020年2月期|2021年2月期|2022年2月期|2023年2月期|2024年2月期|
|---|---|---|---|---|---|---|
|総資産（百万円）|60,284|53,737|53,000|56,884|50,211|50,643|
|純資産（百万円）|43,587|39,588|39,543|42,917|38,214|38,596|
|有利子負債（百万円）|175|82.0|0.0|0.0|0.0|0.0|

# キャッシュフロー

|項目|2019年2月期|2020年2月期|2021年2月期|2022年2月期|2023年2月期|2024年2月期|
|---|---|---|---|---|---|---|
|営業活動によるキャッシュ・フロー（百万円）|1,664|2,433|6,633|1,871|3,103|2,666|
|投資活動によるキャッシュ・フロー（百万円）|△|△|△|△|△|△|
|フリーキャッシュ・フロー（百万円）|7,071|2,506|4,463|1,977|732|1,563|

# 1株当たり情報

|項目|2019年2月期|2020年2月期|2021年2月期|2022

In [66]:
recursive_index

In [80]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

reranker = CohereRerank(top_n=5)

In [81]:
recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
    verbose=True
)

### Setup Baseline

In [82]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["1.pdf"])
base_docs = reader.load_data()
raw_index = VectorStoreIndex.from_documents(base_docs)

raw_query_engine = raw_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
)

In [69]:
query = "第7次中期経営計画のテーマは何ですか？"

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
第7次中期経営計画のテーマは「Challenge for Future」です。

***********New LlamaParse+ Recursive Retriever Query Engine***********
Challenge for Future 未来への挑戦～2030年に向けて～


In [70]:
query = "４℃ホールディングスの2024年2月期の売上高はいくらですか？"

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
13,597億円

***********New LlamaParse+ Recursive Retriever Query Engine***********
394.5億円


In [71]:
query = "４℃ホールディングスグループの連結子会社数はいくつありますか？"

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
18社

***********New LlamaParse+ Recursive Retriever Query Engine***********
7社


In [76]:
query = "４℃ホールディングスの2024年2月期の1株当たり年間配当金はいくらですか？"

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
1株当たり年間配当金は80円です。
Retrieval entering bc8b1fbe-399b-4403-a39c-0eb6d1d0306c: TextNode
Retrieving from object TextNode with query ４℃ホールディングスの2024年2月期の1株当たり年間配当金はいくらですか？
Retrieval entering 4d1d9527-0960-425f-93de-23ff24f22149: TextNode
Retrieving from object TextNode with query ４℃ホールディングスの2024年2月期の1株当たり年間配当金はいくらですか？

***********New LlamaParse+ Recursive Retriever Query Engine***********
83円


In [83]:
query = "2024年2月期の従業員の平均年齢は男女それぞれ何歳ですか？"

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
The average age of male employees for the fiscal year ending in February 2024 is 39 years old, and the average age of female employees is 37 years old.
Retrieval entering 5f3a4a99-c7fa-4ee4-b46f-72c7462cc5c0: TextNode
Retrieving from object TextNode with query 2024年2月期の従業員の平均年齢は男女それぞれ何歳ですか？
Retrieval entering 301b9ec2-77f2-40d9-9547-9f6b036b337a: TextNode
Retrieving from object TextNode with query 2024年2月期の従業員の平均年齢は男女それぞれ何歳ですか？

***********New LlamaParse+ Recursive Retriever Query Engine***********
32.9歳（女性）および42.7歳（男性）


In [84]:
query = "2024年2月期の従業員の平均勤続年数は、男女それぞれ何年ですか？"

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
The average length of service for employees in the fiscal year ending in February 2024 is 6.5 years for male employees and 7.2 years for female employees.
Retrieval entering 301b9ec2-77f2-40d9-9547-9f6b036b337a: TextNode
Retrieving from object TextNode with query 2024年2月期の従業員の平均勤続年数は、男女それぞれ何年ですか？
Retrieval entering b1be35dc-21a7-4b92-8dfb-c2c1f88a8699: TextNode
Retrieving from object TextNode with query 2024年2月期の従業員の平均勤続年数は、男女それぞれ何年ですか？

***********New LlamaParse+ Recursive Retriever Query Engine***********
男性の平均勤続年数は13.3年であり、女性の平均勤続年数は8.7年です。
